In [15]:
from urllib.request import urlopen
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime
import math
import random
from urllib.request import urlopen
import numpy as np

In [16]:
# from .env
client = tweepy.Client()

In [17]:
MAX_PLAYER_INFO = 5

In [34]:
def scrapeFbrefStandardStats():
    url = 'https://fbref.com/en/squads/b8fd03ef/2023-2024/all_comps/Manchester-City-Stats-All-Competitions'

    html = urlopen(url).read().decode('utf-8')

    df = pd.read_html(html)[0]
    df = df.reset_index(drop=True)
    df = df.dropna()

    df.loc[df['Unnamed: 0_level_0',   'Player'] == 'Kyle Walker', ('Unnamed: 2_level_0', 'Pos')] = 'DF'

    pos_order = {'FW': 0, 'MF': 1, 'DF': 2, 'GK': 3}
    df['Pos'] = df[('Unnamed: 2_level_0', 'Pos')].str.split(',').str[0].str.strip().map(pos_order)
    df = df.sort_values(by=[('Pos', ''), ( 'Unnamed: 4_level_0', 'MP')], ascending=[True, False]).reset_index(drop=True)
    df = df[[( 'Unnamed: 0_level_0',   'Player'),
                ( 'Unnamed: 2_level_0',      'Pos'),
                ( 'Unnamed: 4_level_0',       'MP'),
                (        'Performance',      'Gls'),
                (        'Performance',      'Ast')
                ]]
    df.columns = ['Player', 'Pos', 'MP', 'Gls', 'Ast']
    df['Gls'] = df['Gls'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)
    df['Ast'] = df['Ast'].replace([np.inf, -np.inf], np.nan).dropna().astype(int)

    return df

In [35]:
df = scrapeFbrefStandardStats()
df

,Player,Pos,MP,Gls,Ast
0,Erling Haaland,FW,21,19,5
1,Phil Foden,"FW,MF",21,7,6
2,Jeremy Doku,FW,17,3,6
3,Bernardo Silva,"FW,MF",16,3,3
4,Jack Grealish,FW,15,1,2
5,Oscar Bobb,FW,7,0,1
6,Julián Álvarez,MF,21,8,5
7,Rodri,MF,18,4,4
8,Mateo Kovačić,MF,15,0,0
9,Rico Lewis,"MF,DF",12,0,2
